# Advanced Skills
## 1. Downsizing data with a specific ratio
### 1) Introduction
Have you thought that given data with 5 labels, you can also do a binary classification? For example, we have 5 jets: t, q, g, w, z. Now instead of 5 Tagger, we only want to do the T Tagger, which means all the other jets are clustered to not-T. In this case, if we directly use the data, it results in input imbalance: T is only 1/4 of not-T. Thus we need to downsize the data so that elements are relatively balanced.<br><br>
Additionally, for testing, we usually use part of the data, so downsizing is very necessary.



In [2]:
import numpy as np
import h5py
import pandas as pd
from tqdm import tqdm
# TODO
# load the data "samples.h5" and find out how many constituents are labbled as "j_t".
# Hint: if a constituent belongs to t tagger, the value of "j_t" should be 1, otherwise 0.
filePath = "data/samples.h5"
with h5py.File(filePath,'r') as f:
    print(f.keys())
df = pd.read_hdf(filePath, key="data")
count = 0
for x in df['j_t']:
    if x == 1:
        count += 1

df

<KeysViewHDF5 ['data']>


,index,j_ptfrac,j_pt,j_eta,j_mass,j_tau1_b1,j_tau2_b1,j_tau3_b1,j_tau1_b2,j_tau2_b2,...,j1_costheta,j1_costhetarel,j1_e1mcosthetarel,j_g,j_q,j_w,j_z,j_t,j_undef,j_index
0,748753,1.0,716.072144,0.275030,43.071781,16.755413,11.134798,8.281958,2.241382,0.909293,...,0.277526,6.123234e-17,3.030103e+02,0,1,0,0,0,0,100139009
1,748754,1.0,716.072144,0.275030,43.071781,16.755413,11.134798,8.281958,2.241382,0.909293,...,0.273948,-3.914251e-03,9.541489e+01,0,1,0,0,0,0,100139009
2,748755,1.0,716.072144,0.275030,43.071781,16.755413,11.134798,8.281958,2.241382,0.909293,...,0.279832,2.464343e-03,7.914750e+01,0,1,0,0,0,0,100139009
3,748756,1.0,716.072144,0.275030,43.071781,16.755413,11.134798,8.281958,2.241382,0.909293,...,0.279832,2.535141e-03,4.681654e+01,0,1,0,0,0,0,100139009
4,748757,1.0,716.072144,0.275030,43.071781,16.755413,11.134798,8.281958,2.241382,0.909293,...,0.249613,-2.984647e-02,4.159912e+01,0,1,0,0,0,0,100139009
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25980,1538582,1.0,1002.740112,-0.312159,38.346062,12.807201,9.645228,7.466561,0.589523,0.482220,...,-0.432120,-9.766927e-02,9.843818e-01,0,0,1,0,0,0,200015407
25981,1538583,1.0,1002.740112,-0.312159,38.346062,12.807201,9.645228,7.466561,0.589523,0.482220,...,-0.330579,1.216667e-01,2.619303e-01,0,0,1,0,0,0,200015407
25982,1538584,1.0,1002.740112,-0.312159,38.346062,12.807201,9.645228,7.466561,0.589523,0.482220,...,-0.199777,7.689060e-02,4.634206e-02,0,0,1,0,0,0,200015407
25983,1538585,1.0,1002.740112,-0.312159,38.346062,12.807201,9.645228,7.466561,0.589523,0.482220,...,-0.346842,-5.932787e-02,2.937252e-02,0,0,1,0,0,0,200015407


However, we cannot directly split this Series. Instead, we want to split the data by jets.

In [3]:
# TODO
# Find out the distribution of each kind of tagger.
# Hint: for each jet, you only want to count it once.
j_t = 0
for x in df['j_q']:
    if x == 1:
        j_t += 1
j_g = 0
for x in df['j_g']:
    if x == 1:
        j_g += 1
j_w = 0
for x in df['j_w']:
    if x == 1:
        j_w += 1
j_z = 0
for x in df['j_z']:
    if x == 1:
        j_z += 1

        
print(j_t)
print(j_g)
print(j_w)
print(j_z)
print(count)
total = j_t + j_g + j_w + j_z + count
print(total)

3817
7193
4173
3805
6997
25985


Now we have a dictionary in which keys are labels and values are correpsponding jet indices.<br>
Let's take 10 for each jet.

In [4]:
# Example: randomly take 10 w jets.
jet_w = np.unique(df[df['j_w']==1]['j_index'])
chosen_jets = np.random.choice(jet_w, size = 10, replace=False) 
# Replace=False => we won't take an element more than once.
w_10 = df[df["j_index"].isin(chosen_jets)]

# TODO #1
# Create a dataframe with 10 jets for each kind.
# Hint: operate seperately and then concatenate.
jet_t = np.unique(df[df['j_t']==1]['j_index'])
chosen_jets_t = np.random.choice(jet_t, size = 10, replace=False)
t_10 = df[df['j_index'].isin(chosen_jets_t)]

jet_g = np.unique(df[df['j_g']==1]['j_index'])
chosen_jets_g = np.random.choice(jet_g, size = 10, replace=False)
g_10 = df[df['j_index'].isin(chosen_jets_g)]

jet_z = np.unique(df[df['j_z']==1]['j_index'])
chosen_jets_z = np.random.choice(jet_z, size = 10, replace=False)
z_10 = df[df['j_index'].isin(chosen_jets_z)]

jet_q = np.unique(df[df['j_q']==1]['j_index'])
chosen_jets_q = np.random.choice(jet_q, size = 10, replace=False)
q_10 = df[df['j_index'].isin(chosen_jets_q)]

#this is the dataframe with 10 for each jet
jets_50 = pd.concat([w_10, t_10, g_10, z_10, q_10], keys=['j_w', 'j_t', 'j_g', 'j_z', 'j_q'])


# TODO #2
# Bonus: Try to create a dataset with 100 jets for total but with ratio t:q:g:z:w = 3:2:1:0:0.
# Hint: change random size.
chosen_jets_t3 = np.random.choice(jet_t, size = 50, replace=False)
t_50 = df[df['j_index'].isin(chosen_jets_t3)]

chosen_jets_q2 = np.random.choice(jet_q, size = 33, replace=False)
q_33 = df[df['j_index'].isin(chosen_jets_q2)]

chosen_jets_g1 = np.random.choice(jet_g, size = 17, replace=False)
g_17 = df[df['j_index'].isin(chosen_jets_g1)]

jets_t3_q2_g1 = pd.concat([t_50, q_33, g_17], keys=['j_t', 'j_q', 'j_g'])

jets_t3_q2_g1

index  j_ptfrac         j_pt     j_eta      j_mass   j_tau1_b1  \
j_t 462     718281       1.0   952.762756 -0.274848  161.044861  135.429428   
    463     718282       1.0   952.762756 -0.274848  161.044861  135.429428   
    464     718283       1.0   952.762756 -0.274848  161.044861  135.429428   
    465     718284       1.0   952.762756 -0.274848  161.044861  135.429428   
    466     718285       1.0   952.762756 -0.274848  161.044861  135.429428   
...            ...       ...          ...       ...         ...         ...   
j_g 21909  3972298       1.0  1064.847046  0.793023   79.658455   32.174557   
    21910  3972299       1.0  1064.847046  0.793023   79.658455   32.174557   
    21911  3972300       1.0  1064.847046  0.793023   79.658455   32.174557   
    21912  3972301       1.0  1064.847046  0.793023   79.658455   32.174557   
    21913  3972302       1.0  1064.847046  0.793023   79.658455   32.174557   

           j_tau2_b1  j_tau3_b1  j_tau1_b2  j_tau2_b2  ...  j1_costheta  \
j_t 462    77.885185  58.987778  26.671320  10.488195  ...    -0.150591   
    463    77.885185  58.987778  26.671320  10.488195  ...    -0.182918   
    464    77.885185  58.987778  26.671320  10.488195  ...    -0.202176   
    465    77.885185  58.987778  26.671320  10.488195  ...    -0.197950   
    466    77.885185  58.987778  26.671320  10.488195  ...    -0.171760   
...              ...        ...        ...        ...  ...          ...   
j_g 21909  20.104313  18.092073   4.908425   1.374587  ...     0.722330   
    21910  20.104313  18.092073   4.908425   1.374587  ...     0.689761   
    21911  20.104313  18.092073   4.908425   1.374587  ...     0.823469   
    21912  20.104313  18.092073   4.908425   1.374587  ...     0.549385   
    21913  20.104313  18.092073   4.908425   1.374587  ...     0.697544   

           j1_costhetarel  j1_e1mcosthetarel  j_g  j_q  j_w  j_z  j_t  \
j_t 462      6.123234e-17          50.946243    0    0    0    0    1   
    463      1.937911e-14          48.746189    0    0    0    0    1   
    464     -9.423048e-03          43.297157    0    0    0    0    1   
    465     -2.071465e-02          41.361618    0    0    0    0    1   
    466      8.448161e-03          39.007011    0    0    0    0    1   
...                   ...                ...  ...  ...  ...  ...  ...   
j_g 21909    6.383471e-03           0.505437    1    0    0    0    0   
    21910    9.912950e-02           0.201123    1    0    0    0    0   
    21911    3.554792e-01           0.147452    1    0    0    0    0   
    21912   -1.465839e-01           0.161832    1    0    0    0    0   
    21913    5.474383e-02           0.130244    1    0    0    0    0   

           j_undef    j_index  
j_t 462          0  400035403  
    463          0  400035403  
    464          0  400035403  
    465          0  400035403  
    466          0  400035403  
...            ...        ...  
j_g 21909        0     196698  
    21910        0     196698  
    21911        0     196698  
    21912        0     196698  
    21913        0     196698  

[6079 rows x 79 columns]

## 2. N-Constituents of Jets
The amount of constituents varies in jets, ranging from 20 to more than 200. When we classify, we always expect to keep each jet containning the same number of constituents. For example, if we set nConstituents = 40, ranking by the transverse momentum (default), the first 40 constituents will be kept. If constituents in a jet are less than 40, we will use zero-padding.

In [5]:
# TODO
# How many constituents are in the jet with j_index=100139009?
constit = np.unique(df[df['j_index']==100139009])
constit.shape

(676,)

You are right if you got 35. But we want exact 40 slots for each jet, which means we need to do zero-padding for the last 5 slots.

In [6]:
# Example: zero-padding [0,1,2] to [0,1,2,0,0]
array = np.arange(0,3,1)

# Create an zero array with expected shape (1,5).
array_exp = np.zeros(5,dtype=int)           
# copy the corresponding elements
for i,ele in enumerate(array):
    array_exp[i] = ele

In [7]:
# Example: Slice [0,1,2,3,4,5,6,7] to [0,1,2,3,4]
array = np.arange(0,8,1)

# Create an zero array with expected shape (1,5).
array_exp = np.zeros(5,dtype=int)           
# Add the corresponding elements
array_exp = array[:5]+array_exp

Now we know how to deal with the two circumstances: nConstituents is greater or lower than the expected number. First let's try integrate it into one method.

In [8]:
# TODO
# Write a method that can fit this array into the shape (2,5).
# Hint: operate on each row and then combine them. (for loop)
array = [[0,1,2],[0,1,2,3,4,5,6,7]]

def fit_array(arr, rows, cols):
    #this will be array/matrix we return
    array_exp = np.zeros((rows, cols), dtype=int)
    
    for row in range(rows):
        array = np.zeros(cols,dtype=int)
        this_row = arr[row]
        if len(this_row) < cols:
            for col,ele in enumerate(this_row):
                array_exp[row][col] = ele
        else:
            array_exp[row] =  this_row[:cols]+array_exp[row]
        
    return array_exp
        

fit_array(array, 2, 5)

array([[0, 1, 2, 0, 0],
       [0, 1, 2, 3, 4]])

In [9]:
# TODO
# In the next step we will use the method we just learned to trim our data into the desired shape.
# Hint:\
# 1. Use "j_index" to identify jets. 
# 2. The expected output shape is in 3D: (mJets, nConstituents, kColumns), 
#      so the zero array you create would be in size (nConstituents, kColumns)
# 3. loop through jets, operate one by one and then combine.

In [10]:
#find number of kcolumns in dataframe
#grab jet by j_index
#use function made above to grab number of desired constituents


## 3. Jet Clustering & Rotating
### 1) Dependencies
You need Linux to run "pyjet". This is the tutorial to install WSL: [WSL Tutorial](https://github.com/451488975/Anaconda_Setup/blob/master/CPU_with_WSL.ipynb)

Make sure you have:
 - pyjet
<br>

### 2) Get low/high-level features
Sometimes we have data with 4-momenta form, either (e,px,py,pz) or (pT,eta,phi,mass). But we need more, such as etaRel,phiRel ...

In [37]:
# TODO
# Load and observe your data: columns, shape, datatype...
# Answer the questions:
# 1. how many jets are there?
    #1000 jets
# 2. how many constituents are in each jet?
    #200 constituents
# 3. Are all the jets are full? (no zero-paddings)
    #No, not all are full
# 4. What's the datatype for each feature
    #float32
# 5. What features are we gonna use to get transverse momentum, psudorapidity...
filePath = "data/4m_samples.h5"

with h5py.File(filePath,'r') as f:
    print(f.keys())
df = pd.read_hdf(filePath, key="data")
#count = 0
#for x in df['j_t']:
 #   if x == 1:
#        count +=
df[:]['E_0']

<KeysViewHDF5 ['data']>


375    474.071136
377    150.504532
378    251.645386
379    451.566132
380    399.093903
          ...    
843    146.659561
845    522.298645
846     81.376282
847    108.247391
848     83.405449
Name: E_0, Length: 1000, dtype: float32

In [25]:
# TODO
# Load with specific shape (nJets, 4, nConstituents)
# Hint: read through the instruction of pands.read_hdf, 
# you will find a way to take just a part of rows and columns.


In [26]:
# I have to provide the answer for the last question, because all subsequent steps are based on this step. 
def _load (filePath, nJets, nConstituents):
    '''
    Returns:
        momenta: (nJets, 4, nConstituents)
    '''
    cols = ['E_'+str(i) for i in range(nConstituents)]+ ['PX_'+str(i) for i in range(nConstituents)] + ['PX_'+str(i) for i in range(nConstituents)] + ['PY_'+str(i) for i in range(nConstituents)] + ['PZ_'+str(i) for i in range(nConstituents)] + ['is_signal_new']

    df = pd.read_hdf(filePath,key='data',stop=nJets, columns = cols)
    # Take all the 4 momentum from 200 particles in all jets and reshape them into one particle per row
    momenta = df.iloc[:,:-1].to_numpy()
    momenta = momenta.reshape(-1,nConstituents,4)
    nJets = slice(nJets)
    momenta = momenta[nJets, :nConstituents, :]
    momenta = np.transpose(momenta, (0, 2, 1))
    label = df['is_signal_new']
    return momenta, label

In [ ]:
# TODO
# get low level features: pT, eta, phi, ... 
# Try to get as many feature as possible, the answer will be probided after camp.
# Hint: reference page (https://www.lhc-closer.es/taking_a_closer_look_at_lhc/0.momentum)



### 3) Cluster and Rotate
Rotation is performed to remove the stochastic nature of the decay angle relative to the η − φ coordinate system. For two-body decay processes (such as the hadronic decay of a W boson) the direction connecting the axes of the leading two subjets can be rotated until the leading subject is directly above the subleading subjet.
<br><br>
More information about Jet-Image: [Paper](https://arxiv.org/pdf/1407.5675.pdf)

In [27]:
# Example: Clustering
# For each event(jet), we cluster it to get subjets.
# Assuming you already have one piece of jet called "event".
# Unfortunately, it is not runable, due to the undefined parameter "event",
# but if you complete the get features part, at least you have 'pT, eta, phi, mass', 
# you can load your results to run this code.
# Notice that event is one jet, and features should follow the order 'pT, eta, phi, mass'.
from pyjet import cluster
flattened_event = np.core.records.fromarrays( [event[:,0],event[:,1],event[:,2],event[:,3]], names= 'pT, eta, phi, mass' , formats = 'f8, f8, f8,f8')
sequence = cluster(flattened_event, R=R0, p= p)
jets = sequence.inclusive_jets()

NameError: name 'event' is not defined

If you get error about pyjet, please check wheter you are using Linux system. WSL is recommmened.

In [ ]:
# TODO
# Try clustering one piece of jet from your results.

Now we have the four features of subjets. We want to put the leading subjet at the origin, and the subleading subjet at -pi/2


In [28]:
# Example: Rotation
pT = event[:, 0]
eta = event[:, 1]
phi = event[:, 2]
mass = event[:, 3]
# shifts all data with respect to the leading subjet so that
# the Jet Image is centerd at the origin (eta,phi) = (0,0).
def deltaPhi(phi1,phi2):
    # Make sure it in the range (-pi, pi)
    x = phi1-phi2
    while x>= np.pi: x -= np.pi*2.
    while x< -np.pi: x += np.pi*2.
    return x

eta -= jets[0].eta
phi = np.array( [deltaPhi(i,jets[0].phi) for i in phi])

# Rotate the jet image such that the second leading
# subjet is located at -pi/2
s1x, s1y = jets[1].eta -jets[0].eta, deltaPhi(jets[1].phi,jets[0].phi)

theta = np.arctan2(s1y, s1x)
if theta < 0.0:
    theta += 2 * np.pi
def rotate(x, y, a):
    xp = x * np.cos(a) - y * np.sin(a)
    yp = x * np.sin(a) + y * np.cos(a)
    return xp, yp
etaRot, phiRot = rotate(eta, phi, np.pi - theta)

NameError: name 'event' is not defined

In [ ]:
# TODO
# Try to cluster and rotate every jet to get etaRot and phiRot.

### 4) Visualization
Let's review how to plot Jet Image.<br>
In this case, we are provided with data with zero-paddings. We need to eliminate them before ploting, since zeros would lead to nan for eta and phi.

In [ ]:
# TODO
# Find out the indices where pT=0 from the features you get previously.
# Hint: np.where()

In [ ]:
# TODO
# get all the non-zero (etaRot,phiRot,pT) set using conditional slicing.
# And then plot a 2D-histogram.